In [7]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_case as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.CLIPS


In [11]:
models = {
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [12]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

model_phases = dr.CLIPS

label_type = dr.SelfReports.AROUSAL
threshold = "fixed"

x, y = train.Train_CASE.get_case_data(
    metrics, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    # features = {name: metrics for name in models.keys()}
    features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.7249092  0.74578424 0.74899647 0.75567572 0.75603503 0.72753741
 0.74783259 0.75359943 0.75738106 0.76361543 0.74995026 0.75825233
 0.75253823 0.75870998 0.76399647 0.76136835 0.76920121 0.77131329
 0.7721111  0.774135   0.78288532 0.78019337 0.77911049 0.78044231
 0.77992282        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...


One or more of the test scores are non-finite: [0.74906956 0.75802499 0.75886126 0.7634001  0.75876323 0.73969426
 0.74539246 0.75415655 0.75483139 0.75326045 0.7486331  0.75501829
 0.75953143 0.75803995 0.75942778 0.74273407 0.74600655 0.74976686
 0.74467035 0.7454649  0.76450027 0.76280522 0.7656597  0.76665535
 0.76411348 0.75402948 0.7651602  0.76954192 0.76892948 0.77711946
 0.76388097 0.7611682  0.76323049 0.75510342 0.76046501 0.75011756
 0.75003342 0.74687519 0.7488814  0.74858946 0.77808232 0.76226665
 0.75878156 0.75634483 0.75832928 0.74293073 0.74567275 0.7406501
 0.7387028  0.73997313 0.76764253 0.7587304  0.74685036 0.75064334
 0.74658937 0.76895165 0.75653372 0.75666016 0.74996243 0.74466324
 0.75487986 0.76484306 0.7552924  0.75435734 0.75344521 0.76547408
 0.75004274 0.75043198 0.74667846 0.73931714 0.75098237 0.73239291
 0.73459218 0.72920792 0.72866627 0.73533549 0.73619387 0.73418549
 0.73332326 0.73382108        nan        nan        nan        nan
        nan     

LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training LGB ...
Model LGB, Actual: [0 1], [27 18], Predictions: [0 1], [32 13]
Training RF ...
Model RF, Actual: [0 1], [27 18], Predictions: [0 1], [30 15]
Training XGB ...
Model XGB, Actual: [0 1], [27 18], Predictions: [0 1], [35 10]

[('hf_rr', 121), ('sdnn', 113), ('rmssd', 107), ('bpm', 95), ('lf_rr', 65)]


[('rmssd', 0.42503531182733906), ('sdnn', 0.191277745136269), ('hf_rr', 0.17677670025511252), ('bpm', 0.11158721568106983), ('lf_rr', 0.09532302710020973)]


[('hf_rr', 0.47767732), ('sdnn', 0.16263756), ('bpm', 0.13289185), ('lf_rr', 0.

...

LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training LGB ...
Model LGB, Actual: [0 1], [31 14], Predictions: [0 1], [33 12]
Training RF ...
Model RF, Actual: [0 1], [31 14], Predictions: [0 1], [35 10]
Training XGB ...
Model XGB, Actual: [0 1], [31 14], Predictions: [0 1], [37  8]

[('hf_rr', 150), ('sdnn', 143), ('lf_rr', 113), ('rmssd', 107), ('bpm', 95)]


[('hf_rr', 0.39851332531205413), ('bpm', 0.17752936795516083), ('rmssd', 0.16591307750462442), ('sdnn', 0.16464352051497444), ('lf_rr', 0.093400708713186)]


[('hf_rr', 0.4558412), ('bpm', 0.15895627), ('lf_rr', 0.15240009), ('rmssd', 0

One or more of the test scores are non-finite: [0.68325602 0.67750144 0.67552772 0.67069765 0.67725036 0.65400768
 0.68535634 0.695198   0.67893949 0.67775372 0.69392295 0.71329846
 0.71044359 0.69878719 0.69440396 0.68412297 0.70278376 0.70222443
 0.69844317 0.70443949 0.70530524 0.71133547 0.7111092  0.70306209
 0.69730656        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...


One or more of the test scores are non-finite: [0.71995156 0.73229398 0.72354069 0.7147598  0.70732932 0.70958588
 0.71541228 0.71939172 0.70594552 0.6980065  0.71763081 0.71358079
 0.71547047 0.70435333 0.69772034 0.70964953 0.71481607 0.71168729
 0.69635459 0.70004297 0.71459939 0.71163217 0.70105673 0.70165094
 0.70159428 0.71500618 0.6929797  0.6947896  0.69488382 0.69125917
 0.71038332 0.69476089 0.70327126 0.7063628  0.69987312 0.70848925
 0.70124575 0.70259564 0.69840739 0.69743641 0.70911371 0.70560758
 0.70276094 0.70041044 0.6936279  0.71270971 0.70224275 0.69246316
 0.68651536 0.68767974 0.71801216 0.70203469 0.69196671 0.69717042
 0.69842851 0.69835939 0.68784347 0.68454108 0.6799759  0.67861519
 0.68795186 0.68373473 0.68163211 0.678564   0.67643082 0.71406279
 0.70434524 0.6935188  0.6900789  0.69105649 0.68551829 0.68350875
 0.67341849 0.67734718 0.67386832 0.70930117 0.69709569 0.69011848
 0.68860869 0.68895768        nan        nan        nan        nan
        nan    

LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training LGB ...
Model LGB, Actual: [0 1], [29 16], Predictions: [0 1], [35 10]
Training RF ...
Model RF, Actual: [0 1], [29 16], Predictions: [0 1], [34 11]
Training XGB ...
Model XGB, Actual: [0 1], [29 16], Predictions: [0 1], [38  7]

[('rmssd', 114), ('hf_rr', 113), ('sdnn', 104), ('bpm', 95), ('lf_rr', 54)]


[('lf_rr', 0.2921005097831796), ('hf_rr', 0.25559165050002675), ('sdnn', 0.1655418720741512), ('bpm', 0.14756349836127242), ('rmssd', 0.13920246928136995)]


[('hf_rr', 0.73079675), ('lf_rr', 0.09917789), ('rmssd', 0.06715129), ('bpm', 